##**PROJETO FINAL 02: Construção de um RAG utilizando LangChain**

## **CESAR SCHOOL**
* Pós-graduação em Engenharia e Análise de Dados - 2023.2
* **Disciplina: Tópicos Complementares**
* Professor: **Silvan Ferreira**
* Aluno: **Allan Bispo** - apsb@cesar.school

###**OBJETIVO: Construção de um RAG utilizando LangChain**
* Desenvolvimento de um sistema `RAG (Retrieval-Augmented Generation)` utilizando a `biblioteca LangChain`.
* Aspectos:
  * Escolha do Documento: um ou mais documentos, podendo ser PDF, texto, páginas da web etc;
  * Splitting do Documento;
  * Criação de Vector Store;
  * Retrieval;
  * Geração de Respostas.

###**Apresentação do Dataset**
####**Documento(s) xxxxxxxxxx**
* xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
* yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
* https://www.zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz

#### **Importando as bibliotecas e API KEY**

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install lambdata langchain_community langchain_openai langchain_text_splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 4.2 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.2 MB/s eta 0:00:00


In [5]:
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader

In [78]:
# Carregar o documento PDF
file_path_pdf = "/content/bolo_de_rolo.pdf"
loader_pdf = PyPDFLoader(file_path_pdf)
docs_pdf = loader_pdf.load()

In [79]:
# Dividir o texto em chunks
text_splitter_pdf = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits_pdf = text_splitter_pdf.split_documents(docs_pdf)

In [80]:
# Configurar o vetor de recuperação
embeddings = OpenAIEmbeddings()
vectorstore_pdf = FAISS.from_documents(all_splits_pdf, embeddings)
retriever_pdf = vectorstore_pdf.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [81]:
# Consultar o vetor de recuperação
query_pdf = "O bolo de rolo é baiano?"
retrieved_docs_pdf = retriever_pdf.invoke(query_pdf)

In [82]:
# Definindo o template do prompt para o assistente
system_template_pdf = """Você é um assistente para tarefas de perguntas e respostas sobre culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.
Pergunta: {question}
Contexto: {context}
Resposta:
"""
prompt_template_pdf = ChatPromptTemplate.from_template(system_template_pdf)

In [87]:
example_messages_pdf = prompt_template_pdf.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages_pdf.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas sobre culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.\nPergunta: alguma pergunta\nContexto: algum contexto\nResposta:\n')]


In [83]:
llm_pdf = ChatOpenAI(model="gpt-4o-mini")

In [84]:
# Função para formatar os documentos recuperados
def format_docs_pdf(docs_pdf):
    return "\n\n".join(doc.page_content for doc in docs_pdf)

# Configurando a cadeia de RAG
rag_chain_pdf = (
    {"context": retriever_pdf | format_docs_pdf, "question": RunnablePassthrough()}
    | prompt_template_pdf
    | llm_pdf
    | StrOutputParser())

In [23]:
perguntas_pdf = ['Com base no conteúdo, o que significa o bolo de rolo ser finalizado com açúcar de confeiteiro polvilhado por cima?',
'O bolo de rolo foi criado no Brasil por imigrantes franceses, de acordo?',
'Com base no conteúdo, explique por que o bolo de rolo é considerado uma adaptação do bolo português.',
'A massa do bolo de rolo deve ser assada em camadas finas para facilitar o enrolamento?',
'O bolo de rolo é tradicionalmente assado em formas redondas e altas?',
'A origem do bolo de rolo remonta ao período colonial, quando foi trazido pelos holandeses, está correto?'
]

In [86]:
# Iterando sobre cada pergunta e executando a cadeia de RAG para cada uma
for i, pergunta in enumerate(perguntas_pdf):
    print(f"Pergunta {i+1}: {pergunta}")
    for chunk in rag_chain_pdf.stream(pergunta):
        print(chunk, end="", flush=True)
    print("\n")

Pergunta 1: Com base no conteúdo, o que significa o bolo de rolo ser finalizado com açúcar de confeiteiro polvilhado por cima?
O açúcar de confeiteiro polvilhado por cima do bolo de rolo confere um acabamento visual atraente e adiciona um toque doce à sobremesa. Essa finalização também realça o sabor do recheio de goiaba, equilibrando a doçura do bolo.

Pergunta 2: O bolo de rolo foi criado no Brasil por imigrantes franceses, de acordo?
O bolo de rolo é uma adaptação do "colchão de noiva", que era um tipo de pão de ló enrolado com recheio de nozes, trazido pelos colonizadores portugueses. Ao chegar ao Brasil, o recheio foi alterado para goiaba, fruta abundante na região Nordeste.

Pergunta 3: Com base no conteúdo, explique por que o bolo de rolo é considerado uma adaptação do bolo português.
O bolo de rolo é considerado uma adaptação do bolo português "colchão de noiva", que era um pão de ló enrolado com nozes. Os colonizadores substituíram as nozes pela goiaba, fruta abundante no Nord

xxxxxxxxxxxxxxxxVERSÃO COM ENTRADA VIA HTTP XXXXXXXXXXXXXXXXXXXX

In [6]:
!pip install beautifulsoup4
!pip install langchain

In [7]:
# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mc-article-body"))

# Carrega o conteúdo da página
loader_web = WebBaseLoader(
    web_paths=("https://ge.globo.com/pe/futebol/times/santa-cruz/noticia/santa-cruz-inova-na-captacao-de-recursos-e-vende-bolo-de-rolo-para-construir-ct.ghtml",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs_web = loader_web.load()

In [8]:
text_splitter_web = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits_web = text_splitter_web.split_documents(docs_web)

In [9]:
vectorstore_web = FAISS.from_documents(all_splits_web, OpenAIEmbeddings())

In [10]:
retriever_web = vectorstore_web.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs_web = retriever_web.invoke("Qual o time de futebol que obteve ajuda via bolo de rolo?")

In [11]:
# Definindo o template do prompt para o assistente
system_template_web = """Você é um assistente para tarefas de perguntas e respostas sobre futebol, culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.
Pergunta: {question}
Contexto: {context}
Resposta:
"""
prompt_template_web = ChatPromptTemplate.from_template(system_template_web)

In [12]:
example_messages_web = prompt_template_web.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages_web.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas sobre futebol, culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.\nPergunta: alguma pergunta\nContexto: algum contexto\nResposta:\n')]


In [13]:
llm_web = ChatOpenAI(model="gpt-4o-mini")

In [14]:
def format_docs(docs_web):
    return "\n\n".join(doc.page_content for doc in docs_web)

rag_chain_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt_template_web
    | llm_web
    | StrOutputParser()
)

In [21]:
# melhorar e aumentar as perguntas
perguntas_web = ['Qual time de futebol que obteve ajuda via bolo de rolo?',
'O Sport Clube do Recife também adotou a venda de bolo de rolo para ajudar na construção do seu centro de treinamento, correto?',
'De acordo com o conteúdo do site, explique o motivo do Santa Cruz vender bolo de rolo.'
]

In [22]:
# Iterando sobre cada pergunta e executando a cadeia de RAG para cada uma
for i, pergunta in enumerate(perguntas_web):
    print(f"Pergunta {i+1}: {pergunta}")
    for chunk in rag_chain_web.stream(pergunta):
        print(chunk, end="", flush=True)
    print("\n")

Pergunta 1: Qual time de futebol que obteve ajuda via bolo de rolo?
O time de futebol que obteve ajuda via bolo de rolo é o Santa Cruz. A venda do produto visa arrecadar fundos para a construção do seu Centro de Treinamento.

Pergunta 2: O Sport Clube do Recife também adotou a venda de bolo de rolo para ajudar na construção do seu centro de treinamento, correto?
Não sei.

Pergunta 3: De acordo com o conteúdo do site, explique o motivo do Santa Cruz vender bolo de rolo.
O Santa Cruz vende bolo de rolo para arrecadar fundos que serão destinados à construção do seu Centro de Treinamento. A iniciativa visa oferecer um produto acessível aos torcedores enquanto busca minimizar o tempo de espera para a conclusão das obras.



xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxLANGCHAINXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [ ]:
# alterar a entrada para pergunta_1 descobrir na web qual a comida utilizada e na pergunta_2 solicitar a receita que está no pdf.
# usar como base o script abaixo

In [ ]:
prompt_template_1 = ChatPromptTemplate.from_template("Explique de forma simplificada e usando no máximo 20 palavras o {topico} fornecido pelo usuário.")
prompt_template_2 = ChatPromptTemplate.from_template("Traduza a {explicacao} para o inglês de forma direta, sem introdução ou frase adicional.")

chain_1 = prompt_template_1 | llm | parser
chain_2 = prompt_template_2 | llm | parser

def resumo(texto):
    prompt_1 = prompt_template_1.invoke({"topico": texto})
    response_1 = chain_1.invoke(prompt_1)
    print(response_1)
    print()

    explicacao = response_1

    prompt_2 = prompt_template_2.invoke({"explicacao": explicacao})
    response_2 = chain_2.invoke(prompt_2)

    return response_2

In [ ]:
entrada = "A creatina é um suplemento popular entre atletas, auxiliando na produção de energia muscular e melhorando o desempenho em atividades de alta intensidade.\nAtletas de diversas modalidades, como musculação, crossfit e esportes de força, utilizam a creatina para aumentar a força e a resistência muscular.\nA creatina é um composto natural encontrado principalmente na carne e no peixe.\nNo corpo, ela atua como um depósito de energia, fornecendo um impulso extra durante os exercícios."
print(entrada)

A creatina é um suplemento popular entre atletas, auxiliando na produção de energia muscular e melhorando o desempenho em atividades de alta intensidade.
Atletas de diversas modalidades, como musculação, crossfit e esportes de força, utilizam a creatina para aumentar a força e a resistência muscular.
A creatina é um composto natural encontrado principalmente na carne e no peixe.
No corpo, ela atua como um depósito de energia, fornecendo um impulso extra durante os exercícios.


In [ ]:
output = resumo(entrada)

print(output)

A creatina é um suplemento que melhora a energia e o desempenho muscular em atividades intensas.

Creatine is a supplement that improves energy and muscle performance in intense activities.
